# Hướng dẫn chuyển đổi MRI sang CT tổng hợp

Notebook này hướng dẫn cách sử dụng mô hình CycleGAN đã được huấn luyện để chuyển đổi ảnh MRI sang ảnh CT tổng hợp cho mục đích xạ trị.

## Nội dung
1. Thiết lập môi trường
2. Tải mô hình đã huấn luyện
3. Chuẩn bị dữ liệu MRI
4. Chuyển đổi MRI sang CT tổng hợp
5. Hiển thị kết quả
6. Phân đoạn mô dựa trên đơn vị Hounsfield (HU)
7. Lưu kết quả
8. Đánh giá chất lượng chuyển đổi (tùy chọn)

## 1. Thiết lập môi trường

Đầu tiên, chúng ta cần cài đặt và nhập các thư viện cần thiết.

In [ ]:
# Cài đặt các thư viện cần thiết (nếu chưa có)
# !pip install torch torchvision numpy matplotlib pydicom nibabel SimpleITK scikit-image pyyaml

import os
import sys
import yaml
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

# Thêm thư mục gốc vào đường dẫn để có thể nhập các module
sys.path.append('..')

# Nhập các module cần thiết từ dự án
from models.cycle_gan import CycleGANModel
from utils.data_utils import load_dicom_series, save_dicom_series, normalize_image
from utils.visualization import display_slices, compare_images
from utils.config import load_config

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

## 2. Tải cấu hình và mô hình đã huấn luyện

Tiếp theo, chúng ta sẽ tải cấu hình và mô hình CycleGAN đã được huấn luyện.

In [ ]:
# Tải cấu hình
config_path = '../configs/default.yaml'
config = load_config(config_path)

# Đường dẫn đến checkpoint mô hình
checkpoint_path = '../data/output/models/checkpoints/best_model.pth'

# Khởi tạo mô hình
model = CycleGANModel(
    input_channels=config['model']['input_channels'],
    output_channels=config['model']['output_channels'],
    generator_filters=config['model']['generator_filters'],
    discriminator_filters=config['model']['discriminator_filters'],
    n_residual_blocks=config['model']['n_residual_blocks']
)

# Tải trọng số mô hình
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Đã tải mô hình từ epoch {checkpoint['epoch']}")
else:
    print(f"Không tìm thấy checkpoint tại {checkpoint_path}")
    print("Vui lòng huấn luyện mô hình trước hoặc tải mô hình đã huấn luyện")

model.to(device)
model.eval()  # Đặt mô hình ở chế độ đánh giá